# Logistic Regression
- Computing Hypothesis
- Computing Cost Function
- Evaluation
- Higher Implementation

## Hypothesis
$ H(X) = \frac{1}{1+e^{-W^{T}X}} $

## Cost
$ cost(W) = -\frac{1}{m}\sum{ylog(H(x))+(1-y)(log(1-H(x))} $
- if $y \simeq  H(x)$, cost는 0에 수렴
- if $y \neq H(x)$, cost는 high

## Gradient Descent
$W:= W - \alpha\frac{\partial}{\partial W}cost(W) $

## import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [57]:
torch.manual_seed(1)

In [2]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

In [3]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [4]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis

In [7]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [9]:
W = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

$ H(X) = \frac{1}{1+e^{-W^{T}X}} = \frac{1}{1+e^{-X \cdot W}} = \frac{1}{1+e^{-X \cdot W+b}} $

In [13]:
#hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W)+b)))
hypothesis = 1 / (1 + torch.exp(-(torch.matmul(x_train, W)+b))) # 둘 다 같다.

In [14]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


In [15]:
# sigmoid() 함수를 제공하고 있다.
print('1/(1+e^{-1} edquals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1} edquals:  tensor([0.7311])


In [16]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)  # 결과적으로 같다.

In [17]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Computing the Cost Function
$ cost(W) = -\frac{1}{m}\sum{ylog(H(x))+(1-y)(log(1-H(x))} $

In [18]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [19]:
# sample 하나에 대한 계산
-(y_train[0] * torch.log(hypothesis[0])+ 
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

In [20]:
losses = -(y_train * torch.log(hypothesis) + 
  (1 - y_train) * torch.log(1 - hypothesis))

In [21]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [46]:
# cross entropy나 binary 로 계산 (bce로 쓰기도 함)
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.0199, grad_fn=<BinaryCrossEntropyBackward0>)

# Training 

In [62]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr = 1)

nb_epoches = 1000
for epoch in range(nb_epoches + 1):
    
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)  # 결과적으로 같다.
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad() # gradient 초기화
    cost.backward() # gradient 계산
    optimizer.step() # 개선

    if epoch % 100 == 0:
        print('Epoch {:4d}/{}, cost: {:.6f}'.format(epoch, nb_epoches, cost.item()))
  


Epoch    0/1000, cost: 0.693147
Epoch  100/1000, cost: 0.134722
Epoch  200/1000, cost: 0.080643
Epoch  300/1000, cost: 0.057900
Epoch  400/1000, cost: 0.045300
Epoch  500/1000, cost: 0.037261
Epoch  600/1000, cost: 0.031672
Epoch  700/1000, cost: 0.027556
Epoch  800/1000, cost: 0.024394
Epoch  900/1000, cost: 0.021888
Epoch 1000/1000, cost: 0.019852


# Evaluation

In [63]:
hypotheisis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7711e-04],
        [3.1636e-02],
        [3.9014e-02],
        [9.5618e-01],
        [9.9823e-01]], grad_fn=<SliceBackward0>)


In [49]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [50]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [51]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


In [64]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [65]:
model = BinaryClassifier()

In [66]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

W = torch.zeros((2,1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD(model.parameters(), lr = 1)

nb_epoches = 100
for epoch in range(nb_epoches + 1):
    
    hypothesis = model(x_train)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad() # gradient 초기화
    cost.backward() # gradient 계산
    optimizer.step() # 개선

    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{}, cost: {:.6f}, Accuracy: {:2.2f}%'.format(
            epoch, nb_epoches, cost.item(), accuracy * 100))
  

Epoch    0/100, cost: 0.734527, Accuracy: 50.00%
Epoch  100/100, cost: 0.134652, Accuracy: 100.00%
